In [1]:
import pandas as pd
import numpy as np


In [51]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [52]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [53]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [59]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [60]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,228
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [61]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

KeyError: "['Unnamed: 5'] not found in axis"

In [62]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,228
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [63]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Sales'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Sales'])

In [64]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,228
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [65]:
wsp_contract['Total_Revenue'] = wsp_contract['Sales'] * wsp_contract['Wholesale p.']

In [66]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract.apply(lambda row: row['Realized_Mfg_Profit'] / row['Total_Revenue'] if row['Total_Revenue'] != 0 else 0, axis=1)

In [67]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,228,399,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000


In [68]:
average_demand = wsp_contract['Demand'].mean()
wsp_contract['Optimal_Stock'] = average_demand

In [69]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [71]:
potential_demand = np.arange(50, 151)
expected_sales = []

for demand_realization in potential_demand:
    # Set the demand column to the current demand realization
    wsp_contract['Demand'] = demand_realization
    # Calculate sales for the current demand realization
    wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)
    # Calculate the average sales for this demand realization
    average_sales = wsp_contract['Sales'].mean()
    expected_sales.append(average_sales)

# Take the average of expected sales over all demand realizations
expected_sales_avg = np.mean(expected_sales)

# Set the same expected sales value for all rows in the DataFrame
wsp_contract['Expected Sales'] = expected_sales_avg

In [72]:
wsp_contract.head(40)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales
0,1,1,Manufacturer 1,150,8,Retailer 1,60,60,0,62,300,480,0.625000,106.125,-46.125,75.004095
1,1,2,Manufacturer 1,150,7,Retailer 1,80,80,0,64,320,560,0.571429,106.125,-26.125,75.004095
2,1,3,Manufacturer 1,150,7,Retailer 1,80,80,23,0,228,399,0.571429,106.125,-26.125,75.004095
3,1,4,Manufacturer 1,150,9,Retailer 1,51,51,0,98,306,459,0.666667,106.125,-55.125,75.004095
4,1,5,Manufacturer 1,150,8,Retailer 1,60,60,0,75,300,480,0.625000,106.125,-46.125,75.004095
5,1,6,Manufacturer 1,150,8,Retailer 1,60,60,0,34,300,480,0.625000,106.125,-46.125,75.004095
6,1,7,Manufacturer 1,150,11,Retailer 1,0,0,0,143,0,0,0.000000,106.125,-106.125,75.004095
7,1,8,Manufacturer 1,150,9,Retailer 1,51,51,0,15,306,459,0.666667,106.125,-55.125,75.004095
8,1,9,Manufacturer 1,150,12,Retailer 1,0,0,0,123,0,0,0.000000,106.125,-106.125,75.004095
9,1,10,Manufacturer 1,150,8,Retailer 1,60,60,0,13,300,480,0.625000,106.125,-46.125,75.004095
